1) Install and import nedded libraries:

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm 
from sklearn.metrics import plot_confusion_matrix , confusion_matrix
#from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from  nltk.stem import PorterStemmer
#import scikitplot.estimators as esti
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from  wordcloud import WordCloud

import time
#from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import tree
#from sklearn.neighbors import KNeighborsClassifier

2) Read Data set 

In [ ]:
#Read Data set
Data = pd.read_csv("news.csv",encoding='latin-1')
Data = Data[['title' , 'text' , 'label']]
Data

In [ ]:
##change fake and real to 0 and 1 
Data['label']=np.where(Data['label']=='FAKE',0,1)

##plotting real and fake news numbers "to make sure they are almost equal"
plt.figure(figsize=(16,9))
sns.countplot(Data.label)

##checking that there is no null values 
Data.isna().sum()
#Data['text'] = Data['title'] + ' ' + Data['text']
#del Data['text']
del Data['title']

  
###############################################################################

stemmer = PorterStemmer()
stemmer2 = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
def Clean(text):

  # Frist converting all letters to lower case
  text= text.lower()
  
  # removing unwanted digits ,special chracters from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", text).split()) #tags
  text= ' '.join(re.sub("^@?(\w){1,15}$", " ", text).split())
   
  #text= ' '.join(re.sub("â", " ", text).split())
  #text= ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())   #Links
  #text= ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split()) 
  #text= ' '.join(re.sub(r'http\S+', '',text).split())
  
  
  #text= ' '.join(re.sub(r'www\S+', '',text).split())
  #text= ' '.join(re.sub("\s+", " ",text).split()) #Extrem white Space
  #text= ' '.join(re.sub("[^-9A-Za-z ]", "" ,text).split()) #digits 
  text= ' '.join(re.sub('-', ' ', text).split()) 
  text= ' '.join(re.sub('_', ' ', text).split()) #underscore 
  
  #Display available PUNCTUATION for examples for c in string.punctuation:
  #print(f"[{c}]")
  
  # removing stopwards and numbers from STRING library
  table= str.maketrans('', '', string.punctuation+string.digits)
  text = text.translate(table)
  
  # Split Sentence as tokens words 
  tokens = word_tokenize(text)
  
  # converting words to their root forms by STEMMING THE WORDS 
  stemmed2 = [lemmatizer.lemmatize(word) for word in tokens] #Covert words to their actual root
  #stemmed2 = [stemmer2.stem(word) for word in tokens] # Covert words to their rootbut not actual
  
  # Delete each stop words from English stop words
  #words = [w for w in stemmed1 if not w in n_words] #n_words contains English stop words
  words = [w for w in stemmed2 if not w in stop_words] #n_words contains English stop words

  text  = ' '.join(words)
    
  return text
#########################################################

Data.text=[Clean(x) for x in Data.text]
Data.head(10)

In [ ]:
#Visualization of fake news
print("visualization of FAKE news")
fake_data = Data[Data['label'] == 0]
fake_data = ''.join([text for text in fake_data.text])
wordcloud = WordCloud(width=800, height=500, max_font_size=100, max_words=100).generate(fake_data)

plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


print("visualization of REAL news")
#Visualization of real news
real_data = Data[Data['label'] == 1]
real_data = ''.join([text for text in real_data.text])
wordcloud = WordCloud(width=800, height=500, max_font_size=100, max_words=100).generate(real_data)

plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()



In [ ]:
#data preparation

#Splitting data into train and validation
train_x, test_x, train_y, test_y = train_test_split(Data['text'], Data['label'] , shuffle = False)
#X = Data[['text' , 'title']]
#Y = Data['label']
#train_x, test_x, train_y, test_y = train_test_split(X, Y , shuffle = False)
#feature extraction
vect  = TfidfVectorizer(min_df = 5, max_df =0.8, sublinear_tf = True, use_idf = True)
train_vect= vect.fit_transform(train_x)
test_vect = vect.transform(test_x)


In [ ]:
from sklearn.metrics import classification_report


def TFIDFModels(Model,txt):
    model     = Model
    t0        = time.time()
    model.fit(train_vect, train_y)
    t1        = time.time()
    predicted = model.predict(test_vect)
    t2        = time.time()
    time_train= t1-t0
    time_pred = t2-t1
    
    accuracy  = model.score(train_vect, train_y)
    predicted = model.predict(test_vect)
    
    report = classification_report(test_y, predicted, output_dict=True)
    
    print(txt)
    print("Training time: %fs; Prediction time: %fs \n" % (time_train, time_pred))
    print('Accuracy score train set :', accuracy)
    print('Accuracy score test set  :', accuracy_score(test_y, predicted),'\n')
    
    print("confusion matrix for train data")
    plot_confusion_matrix(model , train_vect,train_y)
    plt.show()
    print("confusion matrix for test data")
    plot_confusion_matrix(model , test_vect,test_y)
    plt.show()
    print('\n -------------------------------------------------------------------------------------- \n')
    return_value = [accuracy_score(test_y, predicted) , time_train ]
    return return_value

In [ ]:
print('Models with Tfidf Feature extraction Techniques : \n')
print('************************************************ \n')

naive_bayes_values = NaiveBayes = TFIDFModels(Model=naive_bayes.MultinomialNB(alpha=0.2),txt='Naive Bays Model : \n ')
log_reg_values=LogReg=TFIDFModels(Model=LogisticRegression(),txt='Logistic Regression Model : \n ')
svm_reg_values=svm=TFIDFModels(Model=SupportVectorClassifier,txt='Support Vector Classifier Model : \n ')
dec_tree_values=DecTree=TFIDFModels(Model=tree.DecisionTreeClassifier(),txt='Decision Tree Classifier Model : \n ')


thisdict = {
  "Naive bayes": (naive_bayes_values[0] * 100),
  "Logistic Regression":(log_reg_values[0]* 100) ,
  "SVM": (svm_reg_values[0]*100),
  "Decision Tree": (dec_tree_values[0]*100)
}

thisdict2 = {
  "Naive bayes": (naive_bayes_values[1] ),
  "Logistic Regression":(log_reg_values[1]) ,
  "SVM": (svm_reg_values[1]),
  "Decision Tree": (dec_tree_values[1])
}

Models = list(thisdict.keys())
Accuracy = list(thisdict.values())
Time = list(thisdict2.values())
  
fig = plt.figure(figsize = (15, 10))
 
# creating the bar plot
plt.bar(Models, Accuracy, color ='maroon',
        width = 0.4)
 
plt.xlabel("Different Models")
plt.ylabel("Accuracy of each model")
plt.title("Summarize the classification accuracy")
plt.show()

fig = plt.figure(figsize = (15, 10))
 
# creating the bar plot
plt.bar(Models, Time, color ='maroon',
        width = 0.4)
 
plt.xlabel("Different Models")
plt.ylabel("Time of each model")
plt.title("Summarize the classification training time")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# defining parameter range
param_grid = {'C': [0.1, 10, 100],
'gamma': [1, 0.1,  0.001],
'kernel': ['rbf','linear','poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
#svm_start_time=time.time()
grid.fit(train_vect, train_y)
grid_predictions = grid.predict(test_x)
SVM_acc=accuracy_score(test_y,grid_predictions)*100

In [ ]:

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
'kernel': ['rbf','linear','poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
#svm_start_time=time.time()
grid.fit(train_vect, train_y)
grid_predictions = grid.predict(test_x)
SVM_acc=accuracy_score(test_y,grid_predictions)*100